# LN - Data PP - Stability and efficiency calculations

## Import libraries and data

In [384]:
import sqlite3
import numpy as np
import pandas as pd
import networkx as nx
import itertools
#import matplotlib.pyplot as plt
import time
import pickle

import os
import re
import sys
import io
from itertools import islice
import math

from tqdm.notebook import trange, tqdm
from time import sleep

from dask_cloudprovider import FargateCluster
from dask.distributed import Client
import dask.array as da
import dask
dask.config.set({'distributed.scheduler.allowed-failures': 50}) 

import s3fs

import boto3


In [69]:


# Load Data

# Initiate s3 resource

session = boto3.session.Session()
s3 = session.resource('s3')


# Dataframe

decisions_load = s3.Object(bucket_name='ln-strategy-data', key='LN_channels.csv').get()
decisions_df=pd.read_csv(io.BytesIO(decisions_load['Body'].read()))

# Channel closures
closure_file = s3.Object(bucket_name='ln-strategy-data', key='channel_closures.p').get()
channel_closures = pickle.loads(closure_file['Body'].read())
    
    
# Channel openings 
opens_file = s3.Object(bucket_name='ln-strategy-data', key='channel_opens.p').get()
channel_opens = pickle.loads(opens_file['Body'].read())

    

# Create list with graph keys

#TODO: Save graphs as numpy array in single H5 file to reduce. Test if creating graphs takes longer than reading from S3

# graph_dir='./data/graph_snapshots' - For local tests
extraction_id=1585344554
graph_keys = [obj.key 
        for obj in s3.Bucket(name='ln-strategy-data').objects.all()
        if re.match(".*"+str(extraction_id)+"_connected/.*\.gpickle",obj.key)]





In [148]:
# Test: extracted formats
print("Number of graph keys:{}".format(len(graph_keys)))
print("---Sample graph keys---")
print(graph_keys[0])
print("---Sample channel opens---")
print(channel_opens[513675])
print("---Sample channel closures---")
print(channel_closures[592638])
print("----Sample blocks----")
print(blocks[1])


Number of graph keys:3996
---Sample graph keys---
graph_snapshots/1585344554_connected/505149.gpickle
---Sample channel opens---
[(5474, 7365, {'capacity': 50000, 'open_fee': 306, 'dec_id': 1, 'channel_id': '513675x2245x0'})]
---Sample channel closures---
[(3098, 1492, {'close_type': 'mutual', 'dec_id': 0, 'channel_id': '505149x622x0'}), (2104, 3098, {'close_type': 'force', 'dec_id': 26620, 'channel_id': '570913x720x1'})]
----Sample blocks----
525952


In [191]:
decisions_df.sort_values(by=['close_block'],inplace=True,ascending=True)
decisions_df.head(10)

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id,bet_mar_node0,bet_mar_node1
1027,535029x2012x1,535029,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,bc1qszamn0la3yqrqhjj8yepdxkl9qlr84zfwgg9zrkccl...,535029.0,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,022a7809052db05fde648391a53aba82286e4a517cff1d...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,462124,2019-07-18 02:48:37,...,NaN,unused,0.0,275630.0,0.0,1027,3160,263,NaN,NaN
1045,535177x446x1,535177,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,bc1qauzljedtlva73ngg7suqketlvn5gnnuemxpeuevcqt...,535177.0,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,02272bd12e59324d0f2b231fb88f134b57eb26dd100d2c...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,257307,2019-07-18 02:48:40,...,NaN,unused,0.0,218405.0,0.0,1045,2096,263,NaN,NaN
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,1.547494e+09,unused,0.0,400000.0,0.0,2745,908,3654,-0.010145,0.001177
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,1.547503e+09,unused,0.0,400000.0,0.0,2744,6052,3654,0.018342,0.014761
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,NaN,unused,0.0,8974.0,0.0,40227,5247,696,-0.019278,-0.006038
37095,562592x1695x1,562592,06b4d9b3cfa10bd2cd33131d034a6b38c1651eee49018a...,bc1q9jnkm78y45kyasnu43p52gc8sqazwy4yfjtqzkwqx5...,562592.0,06b4d9b3cfa10bd2cd33131d034a6b38c1651eee49018a...,029b71b8186914267ea59cb081c43ad1aeb874b5a185a4...,03864ef025fde8fb587d989186ce6a4a186895ee44a926...,5000000,2019-06-13 02:13:01,...,NaN,unused,0.0,4995006.0,0.0,37095,4405,6802,NaN,NaN
37213,564495x455x1,564495,8a5764e1f0cb659b687a0675cd88983526bd9213665986...,bc1qqgcvu4nl3vjm4vvr9r5l2f4ufppph5fn070fzscq4r...,564510.0,655c3d44c09055e9af9f8d13d55c99979e0b0b306230b9...,024655b768ef40951b20053a5c4b951606d4d86085d512...,0375a154b8f94eb0556566d60d96acc47f99f2f0d74ef9...,400000,2019-06-03 04:39:32,...,NaN,unused,0.0,395111.0,0.0,37213,4528,4646,NaN,NaN
37209,564476x2292x0,564476,0e00d6dc5cf2232d15750bd3177c57521cdff678a5666a...,bc1qec377ms3a79e3v3pe8gfjrzp6syfqykeqkw2lsh86f...,564511.0,58fea309da14892858be78c8c45a7d06fa6796e77e51d8...,028303182c9885da93b3b25c9621d22cf34475e63c1239...,03820e3b7bdbf7ccafe67791088de15df162b352f3b7ba...,20000,2019-06-03 04:39:12,...,NaN,unused,0.0,17111.0,0.0,37209,6190,6834,NaN,NaN
37256,564948x2100x0,564948,e1b4de87949168dafae980bc1f467b2bee878c2667383a...,bc1ql47jp0hprvcq4dz8y5dpape60vhf532uea2rv9v20q...,565057.0,af2a36eb0782958a3f2e7aacb89dfdb92defeaed44df3c...,02529db69fd2ebd3126fb66fafa234fc3544477a23d509...,02e63d3e5a2351cc8de6c63b0d0784d1940406c5addce4...,47882,2019-06-03 04:43:42,...,NaN,force,0.0,44239.0,0.0,37256,7699,4242,NaN,NaN
37255,564948x2082x0,564948,eb12d66c34e8009c408fa56d948bf87baec4888caddd2d...,bc1q4ksyf7c7jphsmwdj8n944y0mypy7n28mts5tt9zrph...,565057.0,af557d74c148c434156edc034b0a94d97afcda20e8c2b6...,02e63d3e5a2351cc8de6c63b0d0784d1940406c5addce4...,039edc94987c8f3adc28dab455efc00dea876089a120f5...,47882,2019-06-03 04:43:40,...,NaN,force,0.0,44239.0,0.0,37255,4242,6403,NaN,NaN


In [ ]:
sys.getsizeof(decisions_df)

## Connection to AWS - Fargate Clusters

In [314]:
cluster = FargateCluster(n_workers=100,scheduler_timeout='60 minutes',image='dsrincon/dask-graph:nx-scipy-v1')

/opt/conda/lib/python3.7/contextlib.py:119: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)
Task exception was never retrieved
future: <Task finished coro=<_wrap_awaitable() done, defined at /opt/conda/lib/python3.7/asyncio/tasks.py:596> exception=ClientError('An error occurred (ThrottlingException) when calling the RunTask operation (reached max retries: 4): Rate exceeded.')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/asyncio/tasks.py", line 603, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/providers/aws/ecs.py", line 130, in _
    await self.start()
  File "/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/providers/aws/ecs.py", line 193, in start
    while timeout.run():
  File "/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/utils/timeout.py", line 74, in 

RuntimeError: {'tasks': [], 'failures': [{'reason': "You've reached the limit on the number of tasks you can run concurrently"}], 'ResponseMetadata': {'RequestId': 'b089fde6-a000-499e-8bf1-39aefcc9a81c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b089fde6-a000-499e-8bf1-39aefcc9a81c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '111', 'date': 'Mon, 06 Apr 2020 18:52:43 GMT'}, 'RetryAttempts': 0}}

In [315]:
cluster

In [316]:
#client = Client(cluster)
cluster=Client('18.232.49.70:8786')

/opt/conda/lib/python3.7/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

python
+---------------------------+---------------+
|                           | version       |
+---------------------------+---------------+
| client                    | 3.7.3.final.0 |
| scheduler                 | 3.7.4.final.0 |
| tcp://172.31.0.31:38027   | 3.7.4.final.0 |
| tcp://172.31.0.40:41329   | 3.7.4.final.0 |
| tcp://172.31.10.132:36017 | 3.7.4.final.0 |
| tcp://172.31.11.112:45815 | 3.7.4.final.0 |
| tcp://172.31.11.230:46067 | 3.7.4.final.0 |
| tcp://172.31.12.251:38751 | 3.7.4.final.0 |
| tcp://172.31.13.100:38327 | 3.7.4.final.0 |
| tcp://172.31.16.156:35607 | 3.7.4.final.0 |
| tcp://172.31.16.231:35527 | 3.7.4.final.0 |
| tcp://172.31.17.17:40751  | 3.7.4.final.0 |
| tcp://172.31.17.75:44259  | 3.7.4.final.0 |
| tcp://172.31.18.247:38251 | 3.7.4.final.0 |
| tcp://172.31.18.65:33469  | 3.7.4.final.0 |
| tcp://172.31.19.192:43435 | 3.7.4.final.0 |
|

## Extract Graphs for Analysis

In [321]:
# Lazy extract Graphs

# Function for lazy S3 extraction
def load_snapshots(key):
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name='ln-strategy-data', key=key).get()
    G=pickle.loads(response['Body'].read())
    
    return G
    
# Script to create delayed array
graph_snapshots=[]
blocks=[]

for key in graph_keys[700:3900]: # Remove index for full range
    # Create block list from file_names
    block_i=int(key.split(".")[0].split("/")[-1]) 
    blocks.append(block_i)
    
    # Extract graphs
    G=dask.delayed(load_snapshots)(key)
    graph_snapshots.append(G)
   

In [ ]:
# Test Lazy Graph extract
graph_i=dask.compute(graph_snapshots[0])[0]
block=graph_i.graph['block']
print(type(block))

#graph_snapshots=dask.compute(*graph_snapshots)
#block=graph_snapshots[0].graph['block']
    
#print(len(graph_snapshots[5]))
#print(graph_snapshots[3].graph['block'])

# Delayed testing
#results = dask.compute(*futures)
#graphs=dask.compute(*graph_snapshots)

## Stability/Efficiency analysis by utility definition

In order to understand the potential motivations behind each decision we analyze each decission (opening or closure of a channel) independently from the perspective of each of the participants in the decission, which we'll call the node under analysis. For each decission we extract or compute the following information: 



## Betweeness


Betweenness centrality measures how central is a network to the flow of information in a network. In the case of the Lightning Network the higher the betweenness centrality of a node, the more transactions (messages) that are routed through it. In particular, we will use a measure of betweenness centrality defined in (Brandes and Fleischer 2005 - https://link.springer.com/chapter/10.1007/978-3-540-31856-9_44) that models infomation through a network, as electric current, efficiently and not only considering shortest path. This allows us to account for the fact that not all transactions travel through shortes path given that there are fee and capacity considerations.  

### Baseline betweeness

In [322]:
# Distributed betweeness function

snapshot_bet_list_fut=[]

def bet_cent(g):
   
    #Uncomment depending on approx or full
    #g_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g,weight='capacity',kmax=10000)
    g_bet=nx.algorithms.centrality.current_flow_betweenness_centrality(g,weight='capacity')
    block=g.graph['block']
    #else:
    #g_bet={}
    return (block,g_bet)
    

for g in graph_snapshots:
    block_bet_tuple=dask.delayed(bet_cent)(g)
    snapshot_bet_list_fut.append(block_bet_tuple)

futures_bet = dask.persist(*snapshot_bet_list_fut)

In [323]:
start=time.time()
snapshot_bet_list = dask.compute(*futures_bet)
end=time.time()
print('Compute in seconds: {}'.format(end-start))

Compute in seconds: 1091.2476680278778


In [324]:
# Define dictionary 
snapshot_bet={record[0]:record[1] for record in snapshot_bet_list}

In [325]:
# Test results and size of betweeness in memory
# Create list with graph keys
print(sys.getsizeof(snapshot_bet))
#n_items = take(10, snapshot_bet.items())
#print(n_items)

147560


### Pairwise stability 

- **Marginal betweenness (bet_mar_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting the decission (adding or removing a channel). Weighted current betweenness centrality is used for this measure.

> **Marginal betweenness for opens** 

In [326]:
#------STABILITY FOR OPENS----

# Function to calculate marginal betweenness centrality for all channel openings in snapshot

def bet_mar_open(input_tuple):
    
    block=input_tuple[0]
    G=input_tuple[1]
    block_opens=input_tuple[2]
    G_bets=input_tuple[3]
  

    # For each open calculate marginal betweenness for each node in channel
    
    #Dictionaries to store marginal betweeness centrality for nodei
    bet_mar_node0_dic_i={} 
    bet_mar_node1_dic_i={} 
    for open_edge in block_opens:
        
        # Extract info about channel
        channel_id=open_edge[2]['channel_id']
        node0=open_edge[0]
        node1=open_edge[1]
        edge_list=[open_edge]
        
        
        
        # Copy original graph
        g_mar=G.copy()   
        old_nodes=False
        
        
        
        # Retrieve betweenness before channel if nodes existed, else define betweeness as 0
        if (g_mar.has_node(node0)):
            node0_bet=G_bets[node0]
            old_nodes=True
        else:
            node0_bet=0
            
        if (g_mar.has_node(node1)):
            node1_bet=G_bets[node1]
            old_nodes=True
        else:
            node1_bet=0
        
            
        if old_nodes:    
            # Add edges and calculate betweeness if at least one of the nodes is in graph 
            g_mar.add_edges_from(edge_list)
            #Uncomment for approx measure
            #g_mar_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g_mar,weight='capacity',kmax=10000)
            g_mar_bet=nx.algorithms.centrality.current_flow_betweenness_centrality(g_mar,weight='capacity')
            
            node0_mar_bet=(g_mar_bet[node0]-node0_bet)
            node1_mar_bet=(g_mar_bet[node1]-node1_bet)
        else:
            node0_mar_bet=0
            node1_mar_bet=0

        
        # Update dictionary - new betweenness
        bet_mar_node0_dic_i[channel_id]=node0_mar_bet
        bet_mar_node1_dic_i[channel_id]=node1_mar_bet
        
    
    return (bet_mar_node0_dic_i,bet_mar_node1_dic_i)
    

# Script to parallelize bet_mar_open

bet_mar_dicfut=[]
for i in range(1,len(graph_snapshots)):
    
    block=blocks[i]
    block_prev=blocks[i-1]
    block_opens=channel_opens[block]
    #print((block_opens))
    g=graph_snapshots[i-1] # Pass previous graph
    g_bet=snapshot_bet[block_prev]
    input_tuple=(block,g,block_opens,g_bet)
    output_tuple=dask.delayed(bet_mar_open)(input_tuple)
    bet_mar_dicfut.append(output_tuple)

futures_bet_mar = dask.persist(*bet_mar_dicfut)


In [327]:
# Run computation
start=time.time()
bet_mar_diclist = dask.compute(*futures_bet_mar)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_mar_diclist)))


Compute in seconds: 2830.047009706497
Size in memory: 25640


In [328]:
# Test output
print(bet_mar_diclist[10])

({'526199x1530x0': 2.131317791056727e-15}, {'526199x1530x0': 0.0033580327871795823})


In [329]:
# Create single dictionaries for node0 and node1

bet_mar_node0_list=[t[0] for t in bet_mar_diclist]
bet_mar_node1_list=[t[1] for t in bet_mar_diclist]

bet_mar_node0_dic={}
for d in bet_mar_node0_list:
    bet_mar_node0_dic.update(d)
    
bet_mar_node1_dic={}
for d in bet_mar_node1_list:
    bet_mar_node1_dic.update(d)

# Test output
# print(bet_mar_node1_dic)

In [330]:
# Add to DataFrame

# Create empty columns
decisions_df['bet_maropen_node0']=np.nan
decisions_df['bet_maropen_node1']=np.nan

# Populate df with values
decisions_df['bet_maropen_node0']=decisions_df['short_channel_id'].map(bet_mar_node0_dic)
decisions_df['bet_maropen_node1']=decisions_df['short_channel_id'].map(bet_mar_node1_dic)

decisions_df_maropen=decisions_df[decisions_df['bet_maropen_node0'].notnull()]
print(len(decisions_df_filter))


396


In [331]:
decisions_df_maropen.sort_values(by=['open_block'],inplace=True,ascending=True)
decisions_df_filter.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id,bet_mar_node0,bet_mar_node1,bet_marclose_node0,bet_marclose_node1
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,0.0,400000.0,0.0,2745,908,3654,-0.010145,1.176619e-03,-0.008922,0.000211
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,0.0,400000.0,0.0,2744,6052,3654,0.018342,1.476137e-02,0.000272,-0.000155
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,0.0,8974.0,0.0,40227,5247,696,-0.019278,-6.038171e-03,-0.019414,-0.005255
2991,549668x215x0,549668,9db0314ed243141d152057bdb16b6a1fee650cf68c3565...,bc1qjhwn44uyurz7mrc7dwtehzd7rqefzjp9q4zcmhl064...,578710.0,1350e1c9001ec23ca2f58e02da10a3dec69690b031a858...,0217890e3aad8d35bc054f43acc00084b25229ecff0ab6...,02f9eaf9949ca9da663fc24494abae4153ee4f0aee480d...,100000,2019-06-17 04:06:28,...,0.0,79845.0,0.0,2991,7058,3139,0.115050,2.157941e-04,NaN,NaN
2940,549563x2304x0,549563,580895adf54d61e2b0cddc040256be11d865661c866c50...,bc1q42tyxyzw2t8tquk0dm4ffvyrlq776dn336y70s8sa9...,578751.0,df1a986f5315dbe75769b7efba017620f07e659e3e2900...,0223390bea8aa598442d056ace37c07e7364da1341b368...,032d4baebebfdeab7a2ecef2fbe109cbef10de95f05aa5...,20420,2019-06-17 04:06:05,...,0.0,17128.0,0.0,2940,2090,540,-0.000775,3.494538e-02,NaN,NaN
2730,548951x2816x1,548951,9a461d65a7ec250583ae08dc90036670f7038b74d8a2ec...,bc1q6a79t3f606xs3u635z7mge3dkzaxg894p8308qtv0g...,578754.0,09556ea391ed1ad33b9dbb2fafa5c16ce097d69578ba4b...,030c3f19d742ca294a55c00376b3b355c3c90d61c6b6b3...,037eb17d0fee2d20bacea3d78940b40f4ac61a5a7040a2...,1000000,2019-06-17 04:04:38,...,0.0,35060.0,35060.0,2730,218,1070,0.004350,1.835569e-04,NaN,NaN
2915,549558x1279x0,549558,2484708911606f16497a85742460c46e5d77f944d0f6f4...,bc1qsm6crya8ve4pghxf37yhwcwgsp8zke4xv3s25tecvq...,578767.0,0c51829b63ba853ebf0e6e7ba69196c1f5a1a63e0e0dec...,02705407cde2e485542ac068c449d8b2966d46c5506ecb...,032d4baebebfdeab7a2ecef2fbe109cbef10de95f05aa5...,20420,2019-06-17 04:05:56,...,0.0,18865.0,0.0,2915,488,540,-0.000499,4.019322e-04,NaN,NaN
2764,549087x1404x0,549087,d59b04f38f7f9d7b3e520150ec09274301278cffa39378...,bc1q6k4v7xx3w7c7qq65w2xm485qcmfk9sva8u5sh902q0...,578850.0,eaf1ad7a71b21db6c6f9ba66d4e7ff780b29cacb7fcec2...,029a06f84c9bfddcf4dca173a4d1a540e6ce7f75cc7d1d...,03166d4154c205b9eb9a5f6c0b38925c8b2f6fb0dfa088...,200000,2019-06-17 04:04:52,...,0.0,197853.0,0.0,2764,2661,558,-0.000702,-1.123397e-15,NaN,NaN
36592,549070x2528x1,549070,58f8d8f8681072ed4f7426f9a299b962cb955967bcb181...,bc1qmc39hhgyt7wwr5ykhy00r845razf6c5aqpeunv8jan...,578909.0,ea322bbf924e8e191e5d7b20204d7af1c240c8618b4172...,03864ef025fde8fb587d989186ce6a4a186895ee44a926...,039195c0969d69f9a25839cb44bd5e858bd9e397163895...,10000000,2019-06-03 03:25:27,...,0.0,9996402.0,0.0,36592,6802,5142,0.100518,1.872334e-14,NaN,NaN
2947,549568x580x0,549568,f7c6532e0fd71fcd7937c1d2e1c5d892b8d40d6581f8e9...,bc1q087x35h5c7875j648h8mg8tuq862fxyattzqa5n4v9...,578929.0,bcc30f9c9a0621c03e346bd95465cf6053f6b694e343d4...,032d4baebebfdeab7a2ecef2fbe109cbef10de95f05aa5...,033ec518c6ad5b565a89215d9f59c9d6dd2ca3d757b4c8...,20420,2019-06-17 

**TODO**: Why is length of Dataframe longer than the number of snapshots extracted? Could it be that some channels appear more than once in dataframe?

> **Marginal betweenness for closures** 

In [334]:
#------STABILITY FOR CLOSURES----

# Function to calculate marginal betweenness centrality for all channel closures in snapshot

def bet_mar_close(input_tuple):
    
    # Extract input details
    block=input_tuple[0]
    G=input_tuple[1]
    block_closes=input_tuple[2]
    G_bets=input_tuple[3]
   

    # For each closure calculate marginal betweenness for each node in channel
    
    bet_mar_node0_dic_i={} #Dictionary to store marginal betweeness centrality for node0
    bet_mar_node1_dic_i={} #Dictionary to store marginal betweeness centrality for node0
    for close_edge in block_closes:
        
        # Extract info about channel
        channel_id=close_edge[2]['channel_id']
        node0=close_edge[0]
        node1=close_edge[1]
        edge_list=[(node0,node1)]
        
        
        
        # Copy original graph
        g_mar=G.copy() 
        old_nodes=False #If there is at least one old node change to True, to run nx betweeness algo
        
        
        # Retrieve previous betweenness for nodes in snapshot. Check for node in previous graph
        
        if (g_mar.has_node(node0)):
            node0_bet=G_bets[node0]
            old_nodes=True
        else:
            node0_bet=0
        
        if (g_mar.has_node(node1)):
            node1_bet=G_bets[node1]
            old_nodes=True
        else:
            node1_bet=0
        
        # Define marginal graph by removing edges and extracting connected component and calculate marginal betweeness
        
        
        if old_nodes:
            g_mar.remove_edges_from(edge_list)   
            connected_components=[c for c in nx.algorithms.components.connected_components(g_mar)]
            g_mar=g_mar.subgraph(connected_components[0]).copy() 
            # Uncomment for aprox
            #g_mar_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g_mar,weight='capacity',kmax=10000)
            g_mar_bet=nx.algorithms.centrality.current_flow_betweenness_centrality(g_mar,weight='capacity')
        
            # Update marginal values for node0 and node1
        
            if (g_mar.has_node(node0)): #If connected component of marginal graph contains node0 find betweeness
                node0_bet_pos=g_mar_bet[node0]
            else: # else update with fixed value
                node0_bet_pos=0
            
            
            
            if (g_mar.has_node(node1)): #If connected component of marginal graph contains node1 find betweeness
                node1_bet_pos=g_mar_bet[node1]
            else: # else update with fixed value
                node1_bet_pos=0
            
            
        else:
            node0_bet_pos=0
            node1_bet_pos=0
        
        # Calculate final betweeness
        node0_mar_bet=node0_bet_pos-node0_bet
        node1_mar_bet=node1_bet_pos-node1_bet
        
        # Update dictionary - new betweenness
        bet_mar_node0_dic_i[channel_id]=node0_mar_bet
        bet_mar_node1_dic_i[channel_id]=node1_mar_bet
        
    
    return (bet_mar_node0_dic_i,bet_mar_node1_dic_i)
    

# Script to parallelize bet_mar_open

bet_mar_dicfut=[]
with tqdm(total=len(range(1,len(graph_snapshots)))) as pbar:
    for i in range(1,len(graph_snapshots)):
    
        block=blocks[i]
        block_prev=blocks[i-1]
        block_closes=channel_closures[block]
   
        g=graph_snapshots[i-1]
        g_bet=snapshot_bet[block_prev]
        input_tuple=(block,g,block_closes,g_bet)
        output_tuple=dask.delayed(bet_mar_close)(input_tuple)
        bet_mar_dicfut.append(output_tuple)
        pbar.update(1)
        
futures_bet_mar_close = dask.persist(*bet_mar_dicfut)


In [335]:
# Run computation
start=time.time()
bet_mar_diclist_close = dask.compute(*futures_bet_mar_close)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_mar_diclist_close)))


Compute in seconds: 96.1899893283844
Size in memory: 25640


In [300]:
# Test output
print(bet_mar_diclist_close)

(({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({'549037x2737x0': 0.002313189991499001, '549037x2738x0': 0.0006411690078780455}, {'549037x2737x0': 0, '549037x2738x0': 0}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({}, {}), ({},

In [336]:
# Create single dictionaries for node0 and node1 closures

bet_marclose_node0_list=[t[0] for t in bet_mar_diclist_close]
bet_marclose_node1_list=[t[1] for t in bet_mar_diclist_close]

bet_marclose_node0_dic={}
for d in bet_marclose_node0_list:
    bet_marclose_node0_dic.update(d)
    
bet_marclose_node1_dic={}
for d in bet_marclose_node1_list:
    bet_marclose_node1_dic.update(d)

# Test output
print(bet_marclose_node0_dic)

{'535029x2012x1': -1.619468448232908e-13, '535177x446x1': 1.099337634813402e-14, '549037x2737x0': 0.0, '549037x2738x0': 0.0, '549489x1194x1': 3.0899761915836876e-17}


In [346]:
# Add to DataFrame

# Create empty columns
decisions_df['bet_marclose_node0']=np.nan
decisions_df['bet_marclose_node1']=np.nan

# Populate df with values
decisions_df['bet_marclose_node0']=decisions_df['short_channel_id'].map(bet_marclose_node0_dic)
decisions_df['bet_marclose_node1']=decisions_df['short_channel_id'].map(bet_marclose_node1_dic)

decisions_df_marclose=decisions_df[decisions_df['bet_marclose_node1'].notnull()]
#print(len(decisions_df_filter))
decisions_df_marclose

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_mar_node0,bet_mar_node1,bet_marclose_node0,bet_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,bet_maropen_node0,bet_maropen_node1
1027,535029x2012x1,535029,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,bc1qszamn0la3yqrqhjj8yepdxkl9qlr84zfwgg9zrkccl...,535029.0,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,022a7809052db05fde648391a53aba82286e4a517cff1d...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,462124,2019-07-18 02:48:37,...,NaN,NaN,-1.619468e-13,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.002350,4.163955e-15
1045,535177x446x1,535177,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,bc1qauzljedtlva73ngg7suqketlvn5gnnuemxpeuevcqt...,535177.0,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,02272bd12e59324d0f2b231fb88f134b57eb26dd100d2c...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,257307,2019-07-18 02:48:40,...,NaN,NaN,1.099338e-14,0.000000e+00,1.061450e-07,5.605437e-04,NaN,NaN,0.001134,1.120106e-03
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-0.010145,0.001177,0.000000e+00,0.000000e+00,-2.441083e-06,3.004141e-03,NaN,NaN,0.001614,3.589994e-15
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,0.018342,0.014761,0.000000e+00,0.000000e+00,-8.283609e-06,3.004141e-03,NaN,NaN,0.001600,2.819470e-15
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,-0.019278,-0.006038,3.089976e-17,-1.350569e-14,3.089976e-17,-1.350569e-14,NaN,NaN,0.000248,2.496207e-03


- **Actual change in betweenness (bet_act_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted current betweenness centrality is used for this measure.

In [354]:
# Calculate actual difference in betweeness

bet_actopen_node0_dic={}
bet_actopen_node1_dic={}
bet_actclose_node0_dic={}
bet_actclose_node1_dic={}

# List to store tuples of block and node with exceptions
node0_block_ex=[]
node1_block_ex=[]
node0_plus_ex=[]
node1_plus_ex=[]

node0_block_ex_close=[]
node1_block_ex_close=[]
node0_plus_ex_close=[]
node1_plus_ex_close=[]


with tqdm(total=len(range(1,len(graph_snapshots)))) as pbar:
    for i in range(1,len(graph_snapshots)):
        block=blocks[i-1]
        block_plus=blocks[i]
    
    # OPENS: Calculate difference in betweeness for each one
        for open_edge in channel_opens[block_plus]:

            # Extract info about open
            channel_id=open_edge[2]['channel_id']
            node0=open_edge[0]
            node1=open_edge[1]

            # Define base betweeness. If new node, define as 0

            try:
                node0_bet=snapshot_bet[block][node0]

            except KeyError:
                node0_bet=0
                node0_block_ex.append((block,node0))


            try:
                node1_bet=snapshot_bet[block][node1]

            except KeyError:
                node1_bet=0
                node1_block_ex.append((block,node1))


            # Get betweeness after open. If node is not existend, define as 0

            try:
                node0_act_bet=snapshot_bet[block_plus][node0]

            except KeyError:
                node0_act_bet=0
                node0_plus_ex.append((block_plus,node0))

            try:
                node1_act_bet=snapshot_bet[block_plus][node1]

            except KeyError:
                node1_act_bet=0
                node1_plus_ex.append((block_plus,node1))

            # Calculate difference

            node0_act_bet_delta=node0_act_bet-node0_bet
            node1_act_bet_delta=node1_act_bet-node1_bet

            #Update dictionary
            bet_actopen_node0_dic[channel_id]=node0_act_bet_delta
            bet_actopen_node1_dic[channel_id]=node1_act_bet_delta
            

    
    
    # CLOSURES: Calculate difference in betweeness for each one
    
    
        for close_edge in channel_closures[block_plus]:

            # Extract info about open
            channel_id=close_edge[2]['channel_id']
            node0=close_edge[0]
            node1=close_edge[1]

            # Define base betweeness. If new node, define as 0

            try:
                node0_bet=snapshot_bet[block][node0]

            except KeyError:
                node0_bet=0
                node0_block_ex_close.append((block,node0))


            try:
                node1_bet=snapshot_bet[block][node1]

            except KeyError:
                node1_bet=0
                node1_block_ex_close.append((block,node1))


            # Get betweeness after close. If node is not existend, define as 0

            try:
                node0_act_bet=snapshot_bet[block_plus][node0]

            except KeyError:
                node0_act_bet=0
                node0_plus_ex_close.append((block_plus,node0))

            try:
                node1_act_bet=snapshot_bet[block_plus][node1]

            except KeyError:
                node1_act_bet=0
                node1_plus_ex_close.append((block_plus,node1))

            # Calculate difference

            node0_act_bet_delta=node0_act_bet-node0_bet
            node1_act_bet_delta=node1_act_bet-node1_bet

            #Update dictionary
            bet_actclose_node0_dic[channel_id]=node0_act_bet_delta
            bet_actclose_node1_dic[channel_id]=node1_act_bet_delta      
           
        pbar.update(1)
    #TODO: ADJUST BAR

In [ ]:
# OPEN Print blocks and nodes (block,node) that raised exceptions
print("----EXCEPTIONS RAISED----")
print("Node 0 not present in initial block:")
print(node0_block_ex)
print("Node 1 not present in initial block:")
print(node1_block_ex)
print("Node 0 not present in next block:")
print(node0_plus_ex)
print("Node 1 not present in next block:")
print(node1_plus_ex)

In [356]:
# CLOSE Print blocks and nodes (block,node) that raised exceptions
print("----EXCEPTIONS RAISED----")
print("Node 0 not present in initial block:")
print(node0_block_ex_close)
print("Node 1 not present in initial block:")
print(node1_block_ex_close)
print("Node 0 not present in next block:")
print(node0_plus_ex_close)
print("Node 1 not present in next block:")
print(node1_plus_ex_close)

----EXCEPTIONS RAISED----
Node 0 not present in initial block:
[]
Node 1 not present in initial block:
[(535028, 263), (549036, 3654), (549036, 3654)]
Node 0 not present in next block:
[]
Node 1 not present in next block:
[(535029, 263)]


In [ ]:
# Test output
bet_actopen_node1_dic

In [361]:
# Add to DataFrame - OPENS

# Create empty columns
decisions_df['bet_actopen_node0']=np.nan
decisions_df['bet_actopen_node1']=np.nan

# Populate df with values
decisions_df['bet_actopen_node0']=decisions_df['short_channel_id'].map(bet_actopen_node0_dic)
decisions_df['bet_actopen_node1']=decisions_df['short_channel_id'].map(bet_actopen_node1_dic)

decisions_df_actopen=decisions_df[decisions_df['bet_actopen_node0'].notnull()]
#print(len(decisions_df_actopen))
decisions_df_actopen.head(10)

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_mar_node0,bet_mar_node1,bet_marclose_node0,bet_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,bet_maropen_node0,bet_maropen_node1
1027,535029x2012x1,535029,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,bc1qszamn0la3yqrqhjj8yepdxkl9qlr84zfwgg9zrkccl...,535029.0,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,022a7809052db05fde648391a53aba82286e4a517cff1d...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,462124,2019-07-18 02:48:37,...,NaN,NaN,-1.619468e-13,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.002350,4.163955e-15
1045,535177x446x1,535177,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,bc1qauzljedtlva73ngg7suqketlvn5gnnuemxpeuevcqt...,535177.0,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,02272bd12e59324d0f2b231fb88f134b57eb26dd100d2c...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,257307,2019-07-18 02:48:40,...,NaN,NaN,1.099338e-14,0.000000e+00,1.061450e-07,5.605437e-04,NaN,NaN,0.001134,1.120106e-03
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-0.010145,0.001177,0.000000e+00,0.000000e+00,-2.441083e-06,3.004141e-03,NaN,NaN,0.001614,3.589994e-15
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,0.018342,0.014761,0.000000e+00,0.000000e+00,-8.283609e-06,3.004141e-03,NaN,NaN,0.001600,2.819470e-15
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,-0.019278,-0.006038,3.089976e-17,-1.350569e-14,3.089976e-17,-1.350569e-14,NaN,NaN,0.000248,2.496207e-03
3077,550069x2297x1,550069,aed17595eb1f01a97a8b32a22332bd313f141d80820e05...,bc1q2pzvsklfkqly7dzpru2mqwag5x82y34jvprcnyy6zf...,577609.0,484e470941d352721b8925e2394ac337233bb110a642b4...,022c699df736064b51a33017abfc4d577d133f7124ac11...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2000000,2019-06-10 03:35:20,...,NaN,NaN,NaN,NaN,5.119560e-04,1.871720e-03,NaN,NaN,0.000530,5.556153e-04
2577,547872x958x0,547872,80198810730619ad4fcfb6acbe0a752781e7c21ff8d441...,bc1q07sa2nymry0fjqlasezjya25ha0vnrgcrr4qs8yh27...,577610.0,c241cf203365ce261b4b8382e5b1f84fdbe364bc0096c9...,02cdf83ef8e45908b1092125d25c68dcec7751ca8d39f5...,0303a49b5a91f2ee58f7fe539fa026e0e87d84fcf4d3fc...,500002,2019-06-06 18:29:04,...,NaN,NaN,NaN,NaN,1.516082e-03,2.680813e-05,NaN,NaN,0.001516,2.680813e-05
2470,547073x776x0,547073,e7d6054fc69533407f7f731a8ad9b449a865c5ae1fa058...,bc1qux6lrdmh52ynj8hvsphtyu39u4ykd7jx0khhvx4yz5...,577673.0,01cc2d27acd26c32a13e6377331c37c6d1c2b03526ae67...,0214382bdce7750dfcb8126df8e2b12de38536902dc36a...,034955913a44e335be62b4e9ee33be036f72d150302163...,4000000,2019-06-10 03:30:58,...,NaN,NaN,NaN,NaN,4.226350e-04,4.393113e-03,NaN,NaN,0.000423,4.393113e-03
2519,547469x1162x0,547469,19cba59f52878978b0770309a62e5c8a044053ef27f605...,bc1qrqvhhp3qfaxp633rhss55arvq4rhvp0xz9y3kapaqj...,577734.0,33492821fa5c54eb04a874fab39592c9f497df0082b8e9...,0242a4ae0c5bef18048fbecf995094b74bfb0f7391418d...,027b3aa2ddff87263f202275e4ba62cec4590b3f2b8448...,20000,2019-06-10 03:31:20,...,NaN,NaN,NaN,NaN,1.612582e-03,1.165222e-16,NaN,NaN,0.001613,1.005602e-16
330,527710x938x0,527710,4e68311cd01f2b1403fd54e085b613bd01bcfcac24111e...,bc1qwz23r6d

In [378]:
# Add to DataFrame - CLOSE

# Create empty columns
decisions_df['bet_actclose_node0']=np.nan
decisions_df['bet_actclose_node1']=np.nan

# Populate df with values
decisions_df['bet_actclose_node0']=decisions_df['short_channel_id'].map(bet_actclose_node0_dic)
decisions_df['bet_actclose_node1']=decisions_df['short_channel_id'].map(bet_actclose_node1_dic)

decisions_df_actclose=decisions_df[decisions_df['bet_actclose_node0'].notnull()]
#print(len(decisions_df_actopen))
decisions_df_actclose.head(10)

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_mar_node0,bet_mar_node1,bet_marclose_node0,bet_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,bet_maropen_node0,bet_maropen_node1
1027,535029x2012x1,535029,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,bc1qszamn0la3yqrqhjj8yepdxkl9qlr84zfwgg9zrkccl...,535029.0,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,022a7809052db05fde648391a53aba82286e4a517cff1d...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,462124,2019-07-18 02:48:37,...,NaN,NaN,-1.619468e-13,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002350,4.163955e-15
1045,535177x446x1,535177,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,bc1qauzljedtlva73ngg7suqketlvn5gnnuemxpeuevcqt...,535177.0,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,02272bd12e59324d0f2b231fb88f134b57eb26dd100d2c...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,257307,2019-07-18 02:48:40,...,NaN,NaN,1.099338e-14,0.000000e+00,1.061450e-07,5.605437e-04,1.061450e-07,5.605437e-04,0.001134,1.120106e-03
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-0.010145,0.001177,0.000000e+00,0.000000e+00,-2.441083e-06,3.004141e-03,-2.441083e-06,3.004141e-03,0.001614,3.589994e-15
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,0.018342,0.014761,0.000000e+00,0.000000e+00,-8.283609e-06,3.004141e-03,-8.283609e-06,3.004141e-03,0.001600,2.819470e-15
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,-0.019278,-0.006038,3.089976e-17,-1.350569e-14,3.089976e-17,-1.350569e-14,3.089976e-17,-1.350569e-14,0.000248,2.496207e-03


- **Marginal betweeness pairwise stability (bet_mar_pairst/open/close)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a betweenness perspective.

In [401]:
# MARGINAL - Add column with check for pairwise stability compatability using marginal outcomes

# OPEN - Channel is opened if both nodes gain

# Define function 
def bet_pairst_maropen(row):
    if not math.isnan(row['bet_mar_node0']):
        pairst=(row['bet_mar_node0']>=0 and row['bet_mar_node1']>=0)
    else:
        pairst=row['bet_mar_node0']
    return pairst

# Apply function
decisions_df['bet_mar_pairstopen']=decisions_df.apply(bet_pairst_maropen,axis=1)

# CLOSE - Channel is closed if at least one node is better off

# Define function 
def bet_pairst_marclose(row):
    if not math.isnan(row['bet_marclose_node0']):
        pairst=(row['bet_marclose_node0']>0 or row['bet_marclose_node1']>0)
    else:
        pairst=row['bet_marclose_node0']
    return pairst

# Apply function
decisions_df['bet_mar_pairstclose']=decisions_df.apply(bet_pairst_marclose,axis=1)

In [402]:
# Test MARGINAL OPEN
decisions_df[decisions_df['bet_mar_node0'].notnull()][['bet_mar_node0','bet_mar_node1','bet_mar_pairstopen']].head()

,bet_mar_node0,bet_mar_node1,bet_mar_pairstopen
2745,-0.010145,0.001177,False
2744,0.018342,0.014761,True
40227,-0.019278,-0.006038,False
2991,0.115050,0.000216,True
2940,-0.000775,0.034945,False


In [403]:
# Test MARGINAL CLOSE
decisions_df[decisions_df['bet_marclose_node0'].notnull()][['bet_marclose_node0','bet_marclose_node1','bet_mar_pairstclose']].head()

,bet_marclose_node0,bet_marclose_node1,bet_mar_pairstclose
1027,-1.619468e-13,0.000000e+00,False
1045,1.099338e-14,0.000000e+00,True
2745,0.000000e+00,0.000000e+00,False
2744,0.000000e+00,0.000000e+00,False
40227,3.089976e-17,-1.350569e-14,True


- **Actual betweeness pairwise stability (bet_act_pairstopen/close)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a betweenness perspective. 

In [404]:
# ACTUAL - Add column with check for pairwise stability compatability using marginal outcomes

# OPEN - Channel is opened if both nodes gain

# Define function 
def bet_pairst_actopen(row):
    if not math.isnan(row['bet_actopen_node0']):
        pairst=(row['bet_actopen_node0']>=0 and row['bet_actopen_node1']>=0)
    else:
        pairst=row['bet_actopen_node0']
    return pairst

# Apply function
decisions_df['bet_act_pairstopen']=decisions_df.apply(bet_pairst_actopen,axis=1)

# CLOSE - Channel is closed if at least one node is better off

# Define function 
def bet_pairst_actclose(row):
    if not math.isnan(row['bet_actclose_node0']):
        pairst=(row['bet_actclose_node0']>0 or row['bet_actclose_node1']>0)
    else:
        pairst=row['bet_actclose_node0']
    return pairst

# Apply function
decisions_df['bet_act_pairstclose']=decisions_df.apply(bet_pairst_actclose,axis=1)

In [406]:
# Test ACTUAL OPEN
decisions_df[decisions_df['bet_actopen_node0'].notnull()][['bet_actopen_node0','bet_actopen_node1','bet_act_pairstopen']].head()

,bet_actopen_node0,bet_actopen_node1,bet_act_pairstopen
1027,0.000000e+00,0.000000e+00,True
1045,1.061450e-07,5.605437e-04,True
2745,-2.441083e-06,3.004141e-03,False
2744,-8.283609e-06,3.004141e-03,False
40227,3.089976e-17,-1.350569e-14,False


In [407]:
# Test ACTUAL CLOSE
decisions_df[decisions_df['bet_actclose_node0'].notnull()][['bet_actclose_node0','bet_actclose_node1','bet_act_pairstclose']].head()

,bet_actclose_node0,bet_actclose_node1,bet_act_pairstclose
1027,0.000000e+00,0.000000e+00,False
1045,1.061450e-07,5.605437e-04,True
2745,-2.441083e-06,3.004141e-03,True
2744,-8.283609e-06,3.004141e-03,True
40227,3.089976e-17,-1.350569e-14,True


In [410]:
# Save Updated DataFrame to S3

# Create S3 resource and define values
session = boto3.session.Session()
s3 = session.resource('s3')
bucket = 'ln-strategy-data' # already created on S3
csv_buffer = io.StringIO()

# Safe DataFrame
decisions_df.to_csv(csv_buffer)
s3.Object(bucket, 'graph_snapshots/'+str(extraction_id)+'_connected/decisions_df-'+str(extraction_id)+'.csv').put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': 'DFC1B77CE4F44303',
  'HostId': 'xbtGqVxTl9zkJo1V0+1pJnayK8fBEU+mSZlU3RLUQVODE6a0+lqd2/ZZzHQUKxUvfR0soKsqnFs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xbtGqVxTl9zkJo1V0+1pJnayK8fBEU+mSZlU3RLUQVODE6a0+lqd2/ZZzHQUKxUvfR0soKsqnFs=',
   'x-amz-request-id': 'DFC1B77CE4F44303',
   'date': 'Tue, 07 Apr 2020 00:24:06 GMT',
   'etag': '"33f9c7d2fc9eb4b9bcd4b5e23702d944"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"33f9c7d2fc9eb4b9bcd4b5e23702d944"'}


### Nash stability 

- **% Change with respect to not making decision (bet_binstat_deltai)**: The % change in betwewnness centrality, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (bet_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher betweenness centrality. This tells me if my strategy helped me be better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (bet_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher betwneenness centrality. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (bet_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower betwneenness centrality. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average betweenness centrality for all the nodes. 



## Connectivity

### Pairwise stability 

- **Marginal % change in connectivity (con_mar_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting the decission (adding or removing a channel). Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Actual % change in connectivity (con_act_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Marginal connectivity pairwise stability (con_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a connectivity perspective.

- **Actual connectivity pairwise stability (con_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a connectivity perspective.  



### Nash stability 

- **% Change with respect to not making decision (con_binstat_deltai)**: The % change in shortest path average, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (con_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher shortest path average. NOTE: This indicates if the strategy selected made the node better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (con_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher shortest path average. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (con_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower shortest path average. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average shortest path average for all the nodes. 





## Utility Functions

In [93]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))